# Call recording transcription (faster-whisper)

Run this notebook in Google Colab. It installs faster-whisper and ffmpeg, then transcribes one audio file.

## 1. Install dependencies

In [ ]:
!pip install faster-whisper -q
!apt-get install -y ffmpeg -qq

## 2. Imports and upload audio

In [ ]:
import logging
from pathlib import Path
from faster_whisper import WhisperModel
from google.colab import files

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger(__name__)

# Upload one audio file (popup will appear)
uploaded = files.upload()
audio_path = list(uploaded.keys())[0]
log.info("Using audio file: %s", audio_path)

## 3. Load model and transcribe

In [ ]:
log.info("Loading Whisper model (large-v3)...")
model = WhisperModel("large-v3", device="cuda", compute_type="float16")

log.info("Starting transcription: %s", audio_path)
segments, info = model.transcribe(audio_path, beam_size=5)

log.info("Language: %s (prob: %.2f)", info.language, info.language_probability)
log.info("Duration: %.2fs", info.duration)

## 4. Segments and full transcript

In [ ]:
full_text = []
for segment in segments:
    print(f"[{segment.start:.2f}s --> {segment.end:.2f}s] {segment.text}")
    full_text.append(segment.text)

full_transcript = " ".join(full_text).strip()
log.info("Transcription complete. Segments: %d", len(full_text))
if full_transcript:
    log.info("Full text length: %d chars", len(full_transcript))

print("\n--- Full transcript ---")
print(full_transcript)